# Prérequis

Premièrement, nous avons besoin d'importer les bibliothèques python nécessaires pour notre algorithme.

La bibliothèque la plus importante est py_entitymatching, qui est développée par Anhais Group dans le cadre du projet Magellan.

https://sites.google.com/site/anhaidgroup/home

In [ ]:
# importation des bibliothèques requises (notamment py_entitymatching)

!pip install unidecode
!pip install fuzzywuzzy
!pip install missingno
!pip uninstall Cython -y
!pip uninstall py_entitymatching -y
!pip uninstall py_stringsimjoin -y
!pip install py_stringsimjoin==0.3.3
!pip install py_entitymatching==0.4.1


import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np
from fuzzywuzzy import fuzz
from unidecode import unidecode
import missingno as msno
import py_stringsimjoin as ssj
import py_stringmatching as sm
import py_entitymatching as em
import math

Found existing installation: py-entitymatching 0.4.1
Uninstalling py-entitymatching-0.4.1:
  Successfully uninstalled py-entitymatching-0.4.1
Found existing installation: py-stringsimjoin 0.3.3
Uninstalling py-stringsimjoin-0.3.3:
  Successfully uninstalled py-stringsimjoin-0.3.3
  Using cached py_stringsimjoin-0.3.3-cp310-cp310-linux_x86_64.whl


  Using cached py_entitymatching-0.4.1-cp310-cp310-linux_x86_64.whl


--------

Le but des cellules ci dessous est de créer une fonction parsing_address afin de séparer le nom de la voie ("Avenue") du type de la voie ("du Général De Gaulle").

Nous devons avoir une nomenclature homogène entre l'adresse de BAN et de IPE, c'est pourquoi nous séparons le type de voie du nom de voie.

Une autre possibilité de cette fonction est aussi de repérer les abréviations de type de voie et de les convertir en nom complet (Av --> Avenue). Ces abréviations sont listées dans abbreviations_list, et le nom complet de voie dans fullname_list


In [ ]:
abbreviations_list = [
    'ABE', 'ACH', 'AER', 'AIRE', 'ALL', 'ANSE', 'AR', 'ARC', 'ART', 'AUT', 'AV', 'BAL', 'BAS', 'BASE', 'BAT', 'BCH', 'BCLE', 'BD', 'BER', 'BOIS', 'BRC', 'BRE', 'BRG', 'BRTL', 'BSTD',
    'BUT', 'CALE', 'CAMP', 'CAN', 'CAR', 'CARE', 'CARR', 'CASR', 'CAU', 'CC', 'CCAL', 'CD', 'CEIN', 'CGNE', 'CHE', 'CHEM', 'CHEZ', 'CHI', 'CHL', 'CHP', 'CHS', 'CHT', 'CITE', 'CLOS', 'COL',
    'COLI', 'COR', 'CORO', 'COT', 'COTE', 'COTT', 'COUR', 'CPG', 'CR', 'CRS', 'CST', 'CTR', 'CTRE', 'CVE', 'CHV', 'D', 'DIG', 'DOM', 'DSC', 'ECA', 'ECL', 'EGL', 'EN', 'ENC', 'ENV', 'ESC',
    'ESP', 'ESPA', 'FG', 'FON', 'FORM', 'FORT', 'FOS', 'FRM', 'GAL', 'GARE', 'GBD', 'GPE', 'GPL', 'GPT', 'GR', 'GRI', 'HAM', 'HCH', 'HIP', 'HLE', 'HLG', 'HLM', 'ILE', 'ILOT', 'IMM', 'IMP',
    'JARD', 'JTE', 'LD', 'LEVE', 'LOT', 'MAIL', 'MAIS', 'MAR', 'MAS', 'MLN', 'MTE', 'MUS', 'NTE', 'PAE', 'PAL', 'PARC', 'PAS', 'PASS', 'PAT', 'PAV', 'PCH', 'PERI', 'PIM', 'PISTE', 'PKG', 'PL',
    'PLAN', 'PLCI', 'PLE', 'PLN', 'PLT', 'PN', 'PNT', 'PONT', 'PORT', 'POT', 'PR', 'PRE', 'PROM', 'PRQ', 'PRT', 'PRV', 'PTA', 'PTE', 'PTTE', 'QU', 'QUA', 'R', 'RAC', 'RAID', 'RDE', 'REM', 'RES',
    'RLE', 'RN', 'RNG', 'ROC', 'RPE', 'RPT', 'RTD', 'RTE', 'RUELLETTE', 'RUETTE', 'SEN', 'SENTE', 'SQ', 'STA', 'STDE', 'TOUR', 'TPL', 'TRA', 'TRN', 'TRT', 'TSSE', 'VAL', 'VC', 'VCHE', 'VEN', 'VIA',
    'VLA', 'VLGE', 'VOI', 'VR', 'VTE', 'ZA', 'ZAC', 'ZAD', 'ZI', 'ZONE', 'ZUP'
]
#abbreviations = pd.DataFrame({'Abbreviation': abbreviations_list})

In [ ]:
fullname_list = [
    'Abbaye', 'Ancien chemin', 'Aéroport', 'Aire', 'Allée', 'Anse',
    'Ancienne Rue', 'Arcade', 'Ancienne route', 'Autoroute', 'Avenue', 'Balcon',
    'Bastion', 'Base', 'Bâtiment', 'Bas Chemin', 'Boucle', 'Boulevard', 'Berge',
    'Bois', 'Brèche', 'Barrière', 'Bourg', 'Bretelle', 'Bastide', 'Butte', 'Cale',
    'Camp', 'Canal', 'Carrefour', 'Carrière', 'Carrè', 'Caserne', 'Carreau',
    'Chemin communal', 'Centre Commercial', 'Chemin départemental', 'Ceinture',
    'Campagne', 'Chemin', 'Cheminement', 'Chez', 'Charmille', 'Chalet', 'Chapelle',
    'Chaussée', 'Château', 'Cité', 'Clos', 'Col', 'Colline', 'Corniche', 'Coron', 'Côte',
    'Coteau', 'Cottage', 'Cour', 'Camping', 'Chemin rural', 'Cours', 'Castel',
    'Contour', 'Centre', 'Coursive', 'Chemin Vicinal', 'Départementale', 'Digue',
    'Domaine', 'Descente', 'Écart', 'Écluse', 'Église', 'Enceinte', 'Enclos',
    'Enclave', 'Escalier', 'Esplanade', 'Espace', 'Faubourg', 'Fontaine', 'Forum',
    'Fort', 'Fosse', 'Ferme', 'Galerie', 'Gare', 'Grand boulevard', 'Groupe',
    'Grande Place', 'Groupement', 'Grande rue', 'Grille', 'Hameau', 'Haut Chemin',
    'Hippodrome', 'Halle', 'Halage', 'HLM', 'Île', 'Îlot', 'Immeuble', 'Impasse',
    'Jardin', 'Jetée', 'Lieu dit', 'Levée', 'Lotissement', 'Mail', 'Maison', 'Marché',
    'Mas', 'Moulin', 'Montée', 'Musée', 'Nouvelle Route', 'Petite Avenue', 'Palais',
    'Parc', 'Passage', 'Passe', 'Patio', 'Pavillon', 'Petit Chemin', 'Périphérique',
    'Petite Impasse', 'Piste', 'Parking', 'Place', 'Plan', 'Placis', 'Passerelle',
    'Plaine', 'Plateau', 'Passage à Niveau', 'Pointe', 'Pont', 'Port', 'Poterne',
    'Petite Rue', 'Pré', 'Promenade', "Presqu'île", 'Petite Route', 'Parvis',
     'Petite Allée', 'Porte', 'Placette', 'Quai', 'Quartier', 'Rue', 'Raccourci',
     'Raidillon', 'Ronde', 'Rempart', 'Résidence', 'Ruelle', 'Route Nationale',
    'Rangée', 'Rocade', 'Rampe', 'Rond Point', 'Rotonde', 'Route', 'Ruellette',
    'Ruette', 'Sentier', 'Sente', 'Square', 'Station', 'Stade', 'Tour', 'Terre Plein',
    'Traverse', 'Terrain', 'Tertre', 'Terrasse', 'Vallée', 'Voie Communale',
    'Vieux Chemin', 'Venelle', 'Via', 'Villa', 'Village', 'Voie', 'Vieille Rue',
    'Vieille Route', 'Zone Artisanale', 'Zone Aménagement Concerté',
    'Zone Aménagement Différé', 'Zone Industrielle', 'Zone', 'Zone à Urbaniser en Priorité'
]
fullname_list = [string.lower() for string in fullname_list] # Lower Case the fullname of roadtype
fullname_list = [unidecode(string) for string in fullname_list] # Remove accents from words

#abbreviations = pd.DataFrame({'Abbreviation': abbreviations_list})

In [ ]:
abbreviation_to_type_mapping = dict(zip(abbreviations_list, fullname_list))

In [ ]:
def find_string_contains_multi_words(string_list, nb_words):
  #This function to filter the full name of road type that have more than 1 word
  result_list = []
  for string in string_list:
    words = string.split()
    if len(words) == nb_words:
      result_list.append(string)
  return result_list

find_string_contains_2_words =  find_string_contains_multi_words(fullname_list, 2)
find_string_contains_3_words =  find_string_contains_multi_words(fullname_list, 3)


def compare_2_strings(string_list, words, threshold = 75):
  # Calculate the similarity score using Fuzzy Logic of the selected words with each string of the list
  # We just take into account the score that is higher than threshold
  # The highest score corresponds to the word considered to be the best match in the list
  # If no scores exceed the threshold, return None
  best_match = None
  best_score = 0
  for item in string_list:
        similarity_score = fuzz.ratio(words, item)
        if similarity_score >= threshold and similarity_score > best_score:
          best_match = item
          best_score = similarity_score
  return best_match, best_score

def parsing_address(input_string, words_to_compare = 3):
  # This is the recursive function

  words = input_string.split() #Split the address into list of string

  if len(words) > 1:
    nb_words = min(words_to_compare, len(words) - 1)  #Take into account the number of words to compare
  else:
    return '',input_string
  compared_words =' '.join(words[:nb_words])

  if words[0] == "le" or words[0] == "la": # Address start with le/la easily confuse with ile, villa
    return '', input_string                # So we assign these addresses to Nom de voie

  if nb_words == 1:
  #Create the stop condition for recursive function (the base case)
  # In case we need to compare first 1 word of the address, we will compare it with string in abbreviation list and fullname_list
  # After having compared that word with 2 lists, we will see that the word is more similar to which string in which list (to Return it correctly)
    match_abbre, match_score_abbre = compare_2_strings(abbreviations_list, compared_words.upper()) # Uppercase the word when compare with abbreviation list because the string in this list is uppercased
    match_full , match_score_full = compare_2_strings(fullname_list, unidecode(compared_words.lower())) # Lowercase the word and then remove the accents
    if match_abbre != None or match_full != None:
      if match_score_abbre > match_score_full:
        return abbreviation_to_type_mapping[match_abbre], ' '.join(words[1:])
      else:
        return match_full,  ' '.join(words[1:])
    else:
      return '', input_string # If no string in both lists matches that word then Invalid Address or there 's no type of the road in this address

  else:
     # If type of the roads contain 2 or 3 words then
     # we need to compare the first 2,3 words of the input string with the string in fullname_list
     # If Match, stop the function immediately
     # If not, decrease the nb of words for comparison from 3 to 2 or 2 to 1.
    reference_list = find_string_contains_2_words if nb_words == 2 else find_string_contains_3_words
    match_word, match_score = compare_2_strings(reference_list, unidecode(compared_words.lower()))
    if match_word == None :
      return parsing_address(input_string, nb_words - 1)
    else:
      return match_word,  input_string.replace(compared_words,"")

Ainsi, la fonction parsing_address prend en entrée un élément de la colonne "nom_voie" de BAN et le convertit en élément de "TypeVoieImmeuble" et "NomVoieImmeuble" de IPE.

---------

# Preprocessing

Dans cette partie, nous allons devoir obtenir un schéma de données similaire entre la table BAN et la table IPE.
Ainsi, nous devons avoir les mêmes colonnes (notamment séparer le type de voie du nom de voie dans BAN) ainsi qu'avoir des noms de colonnes identiques.

De plus, nous devons nous assurer que le contenu des données est similaires, et donc traiter les majuscules, accents, cartactères spéciaux.

In [ ]:
ipe = pd.read_csv('/content/IPE-AXTD.csv', delimiter=";")
ban = pd.read_csv('/content/BAN-adresses-12.csv', delimiter=";")
# we take ban-adresses-12 as it is the department 12, which is contianed in ipe AXTD region

In [ ]:
# ban['TypeVoie'] = ban['nom_voie'].apply(lambda x : parsing_address(x)[0])
# ban['NomVoieReste'] = ban['nom_voie'].apply(lambda x : parsing_address(x)[1])  #We shouldn't write like this because it will double the processing time

for i in range(len(ban)):
    parsed_address = parsing_address(ban.loc[i, 'nom_voie'])
    #print(i)
    #if parsed_address is not None:
    ban.loc[i, 'TypeVoie'] = parsed_address[0]
    ban.loc[i, 'NomVoieReste'] = parsed_address[1]
ban

,id,id_fantoir,numero,rep,nom_voie,code_postal,code_insee,nom_commune,code_insee_ancienne_commune,nom_ancienne_commune,x,y,lon,lat,type_position,alias,nom_ld,libelle_acheminement,nom_afnor,source_position,source_nom_voie,certification_commune,cad_parcelles,TypeVoie,NomVoieReste
0,12002_0puwwe_00001,NaN,1,NaN,Plan du Foirail,12520,12002,Aguessac,NaN,NaN,707758.86,6339740.77,3.097003,44.156631,entrée,NaN,NaN,AGUESSAC,PLAN DU FOIRAIL,inconnue,inconnue,0,NaN,plan,du Foirail
1,12002_0puwwe_00003,NaN,3,NaN,Plan du Foirail,12520,12002,Aguessac,NaN,NaN,707754.93,6339752.98,3.096954,44.156741,entrée,NaN,NaN,AGUESSAC,PLAN DU FOIRAIL,inconnue,inconnue,0,NaN,plan,du Foirail
2,12002_0puwwe_00005,NaN,5,NaN,Plan du Foirail,12520,12002,Aguessac,NaN,NaN,707758.99,6339764.54,3.097005,44.156845,entrée,NaN,NaN,AGUESSAC,PLAN DU FOIRAIL,inconnue,inconnue,0,NaN,plan,du Foirail
3,12002_0puwwe_00007,NaN,7,NaN,Plan du Foirail,12520,12002,Aguessac,NaN,NaN,707766.91,6339767.00,3.097104,44.156867,entrée,NaN,NaN,AGUESSAC,PLAN DU FOIRAIL,inconnue,inconnue,0,NaN,plan,du Foirail
4,12002_0puwwe_00009,NaN,9,NaN,Plan du Foirail,12520,12002,Aguessac,NaN,NaN,707763.40,6339758.66,3.097060,44.156792,entrée,NaN,NaN,AGUESSAC,PLAN DU FOIRAIL,inconnue,inconnue,0,NaN,plan,du Foirail
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152678,12006_b167_00018,12006_B167,18,NaN,Mas Nespoulous,12430,12006,Alrance,NaN,NaN,675581.14,6334943.59,2.694943,44.113072,entrée,NaN,NaN,ALRANCE,MAS NESPOULOUS,commune,commune,1,120060000E0619,mas,Nespoulous
152679,12006_b167_00020,12006_B167,20,NaN,Mas Nespoulous,12430,12006,Alrance,NaN,NaN,675674.74,6334846.90,2.696117,44.112205,entrée,NaN,NaN,ALRANCE,MAS NESPOULOUS,commune,commune,1,120060000E0743,mas,Nespoulous
152680,12006_b002_00001,12006_B002,1,NaN,La Baraque de Malet,12430,12006,Alrance,NaN,NaN,675308.07,6334466.23,2.691555,44.108766,entrée,NaN,NaN,ALRANCE,LA BARAQUE DE MALET,commune,commune,1,120060000F0538,villa,Baraque de Malet
152681,12006_b002_00003,12006_B002,3,NaN,La Baraque de Malet,12430,12006,Alrance,NaN,NaN,675282.59,6334438.89,2.691238,44.108519,entrée,NaN,NaN,ALRANCE,LA BARAQUE DE MALET,commune,commune,1,120060000F0537,villa,Baraque de Malet


In [ ]:
ban.rename(columns={'typeVoie': 'TypeVoie'}, inplace=True)
ban.rename(columns={'nomVoie': 'NomVoieReste'}, inplace=True)

ban_reduced = ban[['id','numero','rep','TypeVoie','NomVoieReste','code_postal','nom_commune','x','y']]
ipe_reduced = ipe[['IdentifiantImmeuble','NumeroVoieImmeuble','ComplementNumeroVoieImmeuble','TypeVoieImmeuble','NomVoieImmeuble','CodePostalImmeuble','CommuneImmeuble','CoordonneeImmeubleX','CoordonneeImmeubleY']]
ipe_reduced['TypeVoieImmeuble'] = ipe_reduced['TypeVoieImmeuble'].apply(lambda x : unidecode(x).lower() if pd.isna(x) is False else '')


nom_colonnes = {'id': 'IdentifiantImmeuble','numero' : 'NumeroVoieImmeuble','rep' : 'ComplementNumeroVoieImmeuble','TypeVoie' : 'TypeVoieImmeuble','NomVoieReste' : 'NomVoieImmeuble','code_postal' : 'CodePostalImmeuble','nom_commune' : 'CommuneImmeuble','x' : 'CoordonneeImmeubleX','y' : 'CoordonneeImmeubleY'}
ban_reduced = ban_reduced.rename(columns=nom_colonnes)

In [ ]:
ban_reduced

,IdentifiantImmeuble,NumeroVoieImmeuble,ComplementNumeroVoieImmeuble,TypeVoieImmeuble,NomVoieImmeuble,CodePostalImmeuble,CommuneImmeuble,CoordonneeImmeubleX,CoordonneeImmeubleY
0,12002_0puwwe_00001,1,NaN,plan,du Foirail,12520,Aguessac,707758.86,6339740.77
1,12002_0puwwe_00003,3,NaN,plan,du Foirail,12520,Aguessac,707754.93,6339752.98
2,12002_0puwwe_00005,5,NaN,plan,du Foirail,12520,Aguessac,707758.99,6339764.54
3,12002_0puwwe_00007,7,NaN,plan,du Foirail,12520,Aguessac,707766.91,6339767.00
4,12002_0puwwe_00009,9,NaN,plan,du Foirail,12520,Aguessac,707763.40,6339758.66
...,...,...,...,...,...,...,...,...,...
152678,12006_b167_00018,18,NaN,mas,Nespoulous,12430,Alrance,675581.14,6334943.59
152679,12006_b167_00020,20,NaN,mas,Nespoulous,12430,Alrance,675674.74,6334846.90
152680,12006_b002_00001,1,NaN,villa,Baraque de Malet,12430,Alrance,675308.07,6334466.23
152681,12006_b002_00003,3,NaN,villa,Baraque de Malet,12430,Alrance,675282.59,6334438.89


In [ ]:
ipe_reduced

,IdentifiantImmeuble,NumeroVoieImmeuble,ComplementNumeroVoieImmeuble,TypeVoieImmeuble,NomVoieImmeuble,CodePostalImmeuble,CommuneImmeuble,CoordonneeImmeubleX,CoordonneeImmeubleY
0,IMB/12001/X/0001,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06
1,IMB/12001/X/0002,8,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674437.822388,6.361478e+06
2,IMB/12001/X/0003,6,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674408.962423,6.361491e+06
3,IMB/12001/X/0004,10,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674402.615772,6.361449e+06
4,IMB/12001/X/0005,1,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674523.068416,6.361517e+06
...,...,...,...,...,...,...,...,...,...
283826,IMB/48198/X/00TG,5996,NaN,route,du Mas de la Barque,48800,Villefort,773209.199944,6.371295e+06
283827,IMB/48198/X/00TH,6005,NaN,route,du Mas de la Barque,48800,Villefort,773186.659129,6.371244e+06
283828,IMB/48198/X/00TI,5264,NaN,route,du Mas de la Barque,48800,Villefort,773131.114161,6.371255e+06
283829,IMB/48198/X/00TJ,6033,NaN,route,du Mas de la Barque,48800,Villefort,773114.651880,6.371262e+06



Ne pas éxécuter cette section à chaque fois, car elle prend beaucoup de temps d'exécution. L'utilisation optimale est de traiter une fois les données brutes de BAN et IPE et ensuite d'enregister les données traitées ban_reduced & ipe_reduced dans des CSV pour les réutiliser plus tard si besoin.

In [ ]:
ipe_reduced.to_csv('/content/ipe_reduced.csv')
ban_reduced.to_csv('/content/ban_reduced.csv')

------------

In [158]:
"""
ipe = pd.read_csv('/content/IPE-AXTD.csv', delimiter=";")
ban_reduced = pd.read_csv('/content/ban_decomposed-12.csv')

ban_reduced = ban_reduced[['id','numero','rep','TypeVoie','NomVoieReste','code_postal','nom_commune','x','y']]
nom_colonnes = {'id': 'IdentifiantImmeuble','numero' : 'NumeroVoieImmeuble','rep' : 'ComplementNumeroVoieImmeuble','TypeVoie' : 'TypeVoieImmeuble','NomVoieReste' : 'NomVoieImmeuble','code_postal' : 'CodePostalImmeuble','nom_commune' : 'CommuneImmeuble','x' : 'CoordonneeImmeubleX','y' : 'CoordonneeImmeubleY'}
ban_reduced = ban_reduced.rename(columns=nom_colonnes)

ipe_reduced = ipe[['IdentifiantImmeuble','NumeroVoieImmeuble','ComplementNumeroVoieImmeuble','TypeVoieImmeuble','NomVoieImmeuble','CodePostalImmeuble','CommuneImmeuble','CoordonneeImmeubleX','CoordonneeImmeubleY']]
ipe_reduced['TypeVoieImmeuble'] = ipe_reduced['TypeVoieImmeuble'].apply(lambda x : unidecode(x).lower() if pd.isna(x) is False else '')
"""

#si vous avez enregistré les dataframes de la section précédente sous forme de CSV pour les réutiliser, vous pouvez les charger ici en précisant le chemin des CSV.
ban_reduced = pd.read_csv('/content/ban_reduced.csv')
ipe_reduced = pd.read_csv('/content/ipe_reduced.csv')


--------

# First Solution - Rule Based

Dans cette section, nous utilisons la bibliothèques py_entitymatching afin de trouver les adresses similaires entre les 2 bases de données BAN et IPE.


In [159]:
A = ipe_reduced.copy()
B = ban_reduced.copy()

em.set_key(A, 'IdentifiantImmeuble')
em.set_key(B, 'IdentifiantImmeuble')


shape_A = A.shape
shape_B = B.shape
print(shape_A)
print(shape_B)

(283831, 10)
(152683, 10)


In [160]:
def complement_numero_voie(string) :
  try :
    string = string.lower()
  except :
    pass
  if string == 'bis' :
    string = 'b'
  if string == 'ter' :
    string = 't'
  if string == 'quater' :
    string = 'q'
  return string

A['ComplementNumeroVoieImmeuble'] = A['ComplementNumeroVoieImmeuble'].apply(complement_numero_voie)
B['ComplementNumeroVoieImmeuble'] = B['ComplementNumeroVoieImmeuble'].apply(complement_numero_voie)


### Etape 1 : Blocking

ETAPE 1

On compare certaines colonnes de manière stricte --> technique de **blocking**


C'est à dire que s'il n'y a pas de correspondance stricte sur chacune de ces colonnes entre les 2 adresses comparées, on considère que ce ne sont pas les mêmes adresses.

Les colonnes blockées sont les suivantes :
- **NumeroVoieImmeuble**
- **ComplementNumeroVoieImmeuble**
- **TypeVoieImmeuble**
- **CodePostalImmeuble**

In [161]:
# define the blocker attributes to check and put in 1 column (because AttrEquivalenceBlocker can only handle 1 column comparison)
# for the unknown NumeroVoieImmeuble, we impute with the value 0
A['NumeroVoieImmeuble'] = A['NumeroVoieImmeuble'].fillna(value = 0).astype(int)
print('We imputed', len(A[A['NumeroVoieImmeuble'] == 0]), 'unknown NumeroVoieImmeuble in the IPE dataset')

B['NumeroVoieImmeuble'] = B['NumeroVoieImmeuble'].fillna(value = 0).astype(int)
print('We imputed', len(B[B['NumeroVoieImmeuble'] == 0]), 'unknown NumeroVoieImmeuble in the BAN dataset')

A['CodePostalImmeuble'] = A['CodePostalImmeuble'].fillna(value = 00000).astype(int)
print('We imputed', len(A[A['CodePostalImmeuble'] == 0]), 'unknown CodePostalImmeuble in the IPE dataset')

B['CodePostalImmeuble'] = B['CodePostalImmeuble'].fillna(value = 00000).astype(int)
print('We imputed', len(B[B['CodePostalImmeuble'] == 0]), 'unknown CodePostalImmeuble in the BAN dataset')




A['blocking_data'] = A['NumeroVoieImmeuble'].fillna('').astype(str)+ ' '  +  A['ComplementNumeroVoieImmeuble'].fillna('').astype(str) + ' ' + A['TypeVoieImmeuble'].fillna('').astype(str) + ' ' + A['CodePostalImmeuble'].fillna('').astype(str)
B['blocking_data'] = B['NumeroVoieImmeuble'].fillna('').astype(str)+ ' '  +  B['ComplementNumeroVoieImmeuble'].fillna('').astype(str) + ' ' + B['TypeVoieImmeuble'].fillna('').astype(str) + ' ' + B['CodePostalImmeuble'].fillna('').astype(str)

We imputed 122228 unknown NumeroVoieImmeuble in the IPE dataset
We imputed 10 unknown NumeroVoieImmeuble in the BAN dataset
We imputed 0 unknown CodePostalImmeuble in the IPE dataset
We imputed 0 unknown CodePostalImmeuble in the BAN dataset


In [162]:
attributes = ['IdentifiantImmeuble','NumeroVoieImmeuble','ComplementNumeroVoieImmeuble','TypeVoieImmeuble','NomVoieImmeuble','CodePostalImmeuble','CommuneImmeuble','CoordonneeImmeubleX','CoordonneeImmeubleY']

ab = em.AttrEquivalenceBlocker()

# Use block_tables to apply blocking over two input tables.
C = ab.block_tables(A, B,
                    l_block_attr='blocking_data', r_block_attr='blocking_data',
                    l_output_attrs=attributes,
                    r_output_attrs=attributes,
                    l_output_prefix='l_', r_output_prefix='r_')
C

,_id,l_IdentifiantImmeuble,r_IdentifiantImmeuble,l_NumeroVoieImmeuble,l_ComplementNumeroVoieImmeuble,l_TypeVoieImmeuble,l_NomVoieImmeuble,l_CodePostalImmeuble,l_CommuneImmeuble,l_CoordonneeImmeubleX,l_CoordonneeImmeubleY,r_NumeroVoieImmeuble,r_ComplementNumeroVoieImmeuble,r_TypeVoieImmeuble,r_NomVoieImmeuble,r_CodePostalImmeuble,r_CommuneImmeuble,r_CoordonneeImmeubleX,r_CoordonneeImmeubleY
0,0,IMB/12001/X/0001,12001_0052_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06,4,NaN,impasse,Adrien Vezinhet,12630,Agen-d'Aveyron,673832.68,6362135.56
1,1,IMB/12001/X/0001,12001_0040_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06,4,NaN,impasse,Daude de Prades,12630,Agen-d'Aveyron,673699.44,6362045.91
2,2,IMB/12001/X/0001,12001_0083_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674471.55,6361516.82
3,3,IMB/12001/X/0001,12001_0067_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06,4,NaN,impasse,de la Cascade,12630,Agen-d'Aveyron,677171.63,6360381.52
4,4,IMB/12001/X/0001,12001_0069_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06,4,NaN,impasse,de la Croux,12630,Agen-d'Aveyron,674318.84,6362009.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133860,1133860,IMB/12307/X/00BB,12307_0018_00408,408,NaN,chemin,de Bellevue,12410,Curan,689708.742599,6.344055e+06,408,NaN,chemin,de Bellevue,12410,Curan,689727.62,6344090.54
1133861,1133861,IMB/12307/X/00BC,12307_0039_00064,64,NaN,impasse,du Claux,12410,Curan,688529.268230,6.344332e+06,64,NaN,impasse,du Claux,12410,Curan,688513.51,6344373.61
1133862,1133862,IMB/12307/X/00BG,12307_0039_00064,64,NaN,impasse,du Claux,12410,Curan,688560.571568,6.344372e+06,64,NaN,impasse,du Claux,12410,Curan,688513.51,6344373.61
1133863,1133863,IMB/12307/X/00BH,12307_0084_00349,349,NaN,chemin,de Martials,12410,Curan,689480.028156,6.347026e+06,349,NaN,chemin,de Martials,12410,Curan,689478.05,6347044.43


Le dataframe affiché contient toutes les adresses "candidates" qui ont passé l'étape du blocking (étape 1)

### Etape 2 : Filtre distance

**ETAPE 2**

Nous utilisons les coordonnées pour calculer la distance entre 2 adresses. Si la distance est supérieure à 2000m (on peut augmenter/baisser cette limite), on considère que ce ne sont pas les mêmes adresses car trop éloignées.

In [163]:
def distance_calculator(row):
    # left, right will be of type pandas series

    # get coordinates X Y
    left_x = row['l_CoordonneeImmeubleX']
    left_y = row['l_CoordonneeImmeubleY']
    right_x = row['r_CoordonneeImmeubleX']
    right_y = row['r_CoordonneeImmeubleY']

    distance = math.sqrt((right_x - left_x)**2 + (right_y - left_y)**2)

    return distance


In [164]:
print('old shape', C.shape)
C['distanceXY'] = C[['l_CoordonneeImmeubleX','l_CoordonneeImmeubleY','r_CoordonneeImmeubleX','r_CoordonneeImmeubleY']].apply(lambda row : distance_calculator(row)  ,axis = 1)

# we only take rows where distance <= 5000

C = C[C['distanceXY'] <= 5000]

print('new shape after distance filtering', C.shape)
C

old shape (1133865, 19)
new shape after distance filtering (829209, 20)


,_id,l_IdentifiantImmeuble,r_IdentifiantImmeuble,l_NumeroVoieImmeuble,l_ComplementNumeroVoieImmeuble,l_TypeVoieImmeuble,l_NomVoieImmeuble,l_CodePostalImmeuble,l_CommuneImmeuble,l_CoordonneeImmeubleX,l_CoordonneeImmeubleY,r_NumeroVoieImmeuble,r_ComplementNumeroVoieImmeuble,r_TypeVoieImmeuble,r_NomVoieImmeuble,r_CodePostalImmeuble,r_CommuneImmeuble,r_CoordonneeImmeubleX,r_CoordonneeImmeubleY,distanceXY
0,0,IMB/12001/X/0001,12001_0052_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06,4,NaN,impasse,Adrien Vezinhet,12630,Agen-d'Aveyron,673832.68,6362135.56,869.844613
1,1,IMB/12001/X/0001,12001_0040_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06,4,NaN,impasse,Daude de Prades,12630,Agen-d'Aveyron,673699.44,6362045.91,912.011652
2,2,IMB/12001/X/0001,12001_0083_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674471.55,6361516.82,33.955714
3,3,IMB/12001/X/0001,12001_0067_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06,4,NaN,impasse,de la Cascade,12630,Agen-d'Aveyron,677171.63,6360381.52,2957.680510
4,4,IMB/12001/X/0001,12001_0069_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06,4,NaN,impasse,de la Croux,12630,Agen-d'Aveyron,674318.84,6362009.03,512.126756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133860,1133860,IMB/12307/X/00BB,12307_0018_00408,408,NaN,chemin,de Bellevue,12410,Curan,689708.742599,6.344055e+06,408,NaN,chemin,de Bellevue,12410,Curan,689727.62,6344090.54,40.513252
1133861,1133861,IMB/12307/X/00BC,12307_0039_00064,64,NaN,impasse,du Claux,12410,Curan,688529.268230,6.344332e+06,64,NaN,impasse,du Claux,12410,Curan,688513.51,6344373.61,44.155804
1133862,1133862,IMB/12307/X/00BG,12307_0039_00064,64,NaN,impasse,du Claux,12410,Curan,688560.571568,6.344372e+06,64,NaN,impasse,du Claux,12410,Curan,688513.51,6344373.61,47.087153
1133863,1133863,IMB/12307/X/00BH,12307_0084_00349,349,NaN,chemin,de Martials,12410,Curan,689480.028156,6.347026e+06,349,NaN,chemin,de Martials,12410,Curan,689478.05,6347044.43,18.890709


### Etape 3 : Matching

**ETAPE 3**

On calcule des cores de similarité basé sur la distance de Levensthein.
On calcule sur les colonnes qui peuvent avoir des sources d'erreurs plus complexes, comme des fautes de frappes, abréviations, etc... --> technique de **matching**

- **CommuneImmeuble**
- **NomVoieImmeuble**

Si le score de similarité est supérieure à **70%** pour la communne et **80%** pour le nom de la voie, les adresses sont considérées comme similaires.

In [165]:
def jaccard_similarity_score(string1, string2):
  alphabet_tok = sm.AlphabeticTokenizer()
  token1 = alphabet_tok.tokenize(string1)
  token2 = alphabet_tok.tokenize(string2)
  jaccard = sm.Jaccard()  # Example Jaccard similarity
  return jaccard.get_sim_score(token1, token2)

def levenshtein_similarity_score(string1, string2):
  distance = sm.Levenshtein()
  return distance.get_sim_score(string1.lower().replace('-',' '), string2.lower().replace('-',' '))

# Apply the function to create a new column 'SimilarityScore'
C['SimilarityScoreRue'] = C.apply(lambda x : levenshtein_similarity_score(x['l_NomVoieImmeuble'],x['r_NomVoieImmeuble']), axis=1)
C['SimilarityScoreCommune'] = C.apply(lambda x : levenshtein_similarity_score(x['l_CommuneImmeuble'],x['r_CommuneImmeuble']), axis=1)


In [166]:
#first, we filter the commune similiarity
C_filtered = C[C['SimilarityScoreCommune'] >= 0.70]
#then, we filter the street name similiarity
C_filtered = C_filtered[C_filtered['SimilarityScoreRue'] >= 0.80]
print('From table A : ' ,shape_A[0] , '    rows and table B : ', shape_B[0], '   rows we found ', len(C_filtered) , ' matching adresses')

C_filtered

From table A :  283831     rows and table B :  152683    rows we found  74649  matching adresses


,_id,l_IdentifiantImmeuble,r_IdentifiantImmeuble,l_NumeroVoieImmeuble,l_ComplementNumeroVoieImmeuble,l_TypeVoieImmeuble,l_NomVoieImmeuble,l_CodePostalImmeuble,l_CommuneImmeuble,l_CoordonneeImmeubleX,l_CoordonneeImmeubleY,r_NumeroVoieImmeuble,r_ComplementNumeroVoieImmeuble,r_TypeVoieImmeuble,r_NomVoieImmeuble,r_CodePostalImmeuble,r_CommuneImmeuble,r_CoordonneeImmeubleX,r_CoordonneeImmeubleY,distanceXY,SimilarityScoreRue,SimilarityScoreCommune
2,2,IMB/12001/X/0001,12001_0083_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674471.55,6361516.82,33.955714,1.0,1.0
18,18,IMB/12001/X/0023,12001_0072_00004,4,NaN,impasse,de la Parro,12630,Agen-d'Aveyron,674996.864464,6.361416e+06,4,NaN,impasse,de la Parro,12630,Agen-d'Aveyron,674961.77,6361435.86,40.407734,1.0,1.0
34,34,IMB/12001/X/004Z,12001_0071_00004,4,NaN,impasse,des Palanges,12630,Agen-d'Aveyron,677296.591510,6.360516e+06,4,NaN,impasse,des Palanges,12630,Agen-d'Aveyron,677294.97,6360514.05,2.300102,1.0,1.0
42,42,IMB/12001/X/0052,12001_0067_00004,4,NaN,impasse,de la Cascade,12630,Agen-d'Aveyron,677162.757414,6.360389e+06,4,NaN,impasse,de la Cascade,12630,Agen-d'Aveyron,677171.63,6360381.52,11.633556,1.0,1.0
59,59,IMB/12001/X/005R,12001_0070_00004,4,NaN,impasse,des Lignes,12630,Agen-d'Aveyron,672674.967260,6.361881e+06,4,NaN,impasse,des Lignes,12630,Agen-d'Aveyron,672659.70,6361882.59,15.308695,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133860,1133860,IMB/12307/X/00BB,12307_0018_00408,408,NaN,chemin,de Bellevue,12410,Curan,689708.742599,6.344055e+06,408,NaN,chemin,de Bellevue,12410,Curan,689727.62,6344090.54,40.513252,1.0,1.0
1133861,1133861,IMB/12307/X/00BC,12307_0039_00064,64,NaN,impasse,du Claux,12410,Curan,688529.268230,6.344332e+06,64,NaN,impasse,du Claux,12410,Curan,688513.51,6344373.61,44.155804,1.0,1.0
1133862,1133862,IMB/12307/X/00BG,12307_0039_00064,64,NaN,impasse,du Claux,12410,Curan,688560.571568,6.344372e+06,64,NaN,impasse,du Claux,12410,Curan,688513.51,6344373.61,47.087153,1.0,1.0
1133863,1133863,IMB/12307/X/00BH,12307_0084_00349,349,NaN,chemin,de Martials,12410,Curan,689480.028156,6.347026e+06,349,NaN,chemin,de Martials,12410,Curan,689478.05,6347044.43,18.890709,1.0,1.0


In [167]:
print('We found', len(C_filtered), 'matching adresses')

We found 74649 matching adresses


Ce dataframe C_filtered contient uniquement les adresses de IPE ( à gauche ) et BAN ( à droite ) qui sont considérées comme similaires, avec les score de similarité associés, c'est à dire la confiance avec laquelle on peut affirmer ces résultats.

# Second Solution - Machine learning based

# Training

### Train

In [168]:
import random


# 2000 matchs évidents, 2000 matchs non évidents, 2000 non matchs évidents, 2000 non matchs non évidents

#matchs évidents
training_df = C_filtered[(C_filtered['SimilarityScoreRue'] == 1) & (C_filtered['SimilarityScoreCommune'] == 1)].sample(2000)

#matchs non évidents
training_df = pd.concat([training_df,C_filtered[(C_filtered['SimilarityScoreRue'] < 1) | (C_filtered['SimilarityScoreCommune'] < 1)].sample(1300)], ignore_index= True)
training_df = pd.concat([training_df,C_filtered[(C_filtered['SimilarityScoreRue']) < 1 & (C_filtered['SimilarityScoreCommune'] < 1)].sample(500)], ignore_index= True)
training_df = pd.concat([training_df,C_filtered[C_filtered['distanceXY'] > 500].sample(200)], ignore_index= True)

#on met le label à 1 pour les adresses ci dessus (matching adresses)
training_df['label'] = 1




#non match évidents

#SimilarityScore très éloignés
training_df_0 = C[C['SimilarityScoreRue'] < 0.30].sample(500)
training_df_0 = pd.concat([training_df_0, C[C['SimilarityScoreCommune'] < 0.30].sample(500)], ignore_index= True)

#different NumeroVoieImmeuble
df_tmp = C_filtered.copy()
df_tmp['l_NumeroVoieImmeuble'] = df_tmp['l_NumeroVoieImmeuble'].apply(lambda x : random.randint(0, 300))
training_df_0 = pd.concat([training_df_0, df_tmp.sample(1000)], ignore_index= True)


# non matchs non évidents

#SimilarityScore proches & même numero de voie
training_df_0 = pd.concat([training_df_0,C[(C['SimilarityScoreRue'] < 0.70) & (C['SimilarityScoreRue'] > 0.40)].sample(1000)], ignore_index= True)
training_df_0 = pd.concat([training_df_0,C[(C['SimilarityScoreCommune'] < 0.70) & (C['SimilarityScoreCommune'] > 0.40)].sample(1000)], ignore_index= True)

#on met le label à 0 pour les adresses ci dessus (non matching adresses)
training_df_0['label'] = 0

training_df = pd.concat([training_df,training_df_0], ignore_index= True)

training_df

,_id,l_IdentifiantImmeuble,r_IdentifiantImmeuble,l_NumeroVoieImmeuble,l_ComplementNumeroVoieImmeuble,l_TypeVoieImmeuble,l_NomVoieImmeuble,l_CodePostalImmeuble,l_CommuneImmeuble,l_CoordonneeImmeubleX,l_CoordonneeImmeubleY,r_NumeroVoieImmeuble,r_ComplementNumeroVoieImmeuble,r_TypeVoieImmeuble,r_NomVoieImmeuble,r_CodePostalImmeuble,r_CommuneImmeuble,r_CoordonneeImmeubleX,r_CoordonneeImmeubleY,distanceXY,SimilarityScoreRue,SimilarityScoreCommune,label
0,781951,IMB/12161/X/00ER,12161_5wjc6t_00095,95,NaN,impasse,de la Raynaldie,12330,Mouret,662194.025695,6.380112e+06,95,NaN,impasse,de la Raynaldie,12330,Mouret,662193.68,6380109.79,2.553131,1.000000,1.000000,1
1,1128466,IMB/12300/X/00W6,12300_0265_00063,63,NaN,avenue,du Belvédère,12200,Villefranche-de-Rouergue,623913.410177,6.362485e+06,63,NaN,avenue,du Belvédère,12200,Villefranche-de-Rouergue,623923.01,6362478.67,11.330553,1.000000,1.000000,1
2,1071015,IMB/12253/X/00RS,12253_4e04kw_00028,28,NaN,route,de Saint-Martin,12410,Salles-Curan,681964.736000,6.343466e+06,28,NaN,route,de Saint-Martin,12410,Salles-Curan,682044.02,6343419.45,91.977043,1.000000,1.000000,1
3,320187,IMB/12052/X/00W6,12052_0365_00026,26,NaN,rue,des Fauvettes,12700,Capdenac-Gare,627496.023966,6.386185e+06,26,NaN,rue,des Fauvettes,12700,Capdenac-Gare,627496.21,6386196.09,10.591963,1.000000,1.000000,1
4,332931,IMB/12162/X/005K,12162_bdwif5_00001,1,NaN,rue,Camin Dels Prats,12160,Moyrazès,655292.976366,6.360412e+06,1,NaN,rue,Camin Dels Prats,12160,Moyrazès,655292.89,6360407.23,4.607548,1.000000,1.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,490571,IMB/12215/X/00DD,12138_0080_00011,11,NaN,rue,des Bruyères,12330,St-Christophe-Vallon,653274.299804,6.374931e+06,11,NaN,rue,Droite,12330,Marcillac-Vallon,657385.48,6375126.68,4115.828300,0.250000,0.450000,0
7996,471029,IMB/12215/X/00FJ,12138_0038_00004,4,NaN,rue,des Églantiers,12330,St-Christophe-Vallon,652959.463153,6.374414e+06,4,NaN,rue,du Calvaire,12330,Marcillac-Vallon,657400.03,6375500.73,4571.566113,0.357143,0.450000,0
7997,474211,IMB/12215/X/007P,12138_i7ayp2_00005,5,NaN,rue,du Cayla,12330,St-Christophe-Vallon,653272.174131,6.374797e+06,5,NaN,rue,des Noyers,12330,Marcillac-Vallon,657220.97,6374900.96,3950.164661,0.300000,0.450000,0
7998,255113,IMB/12164/X/00C2,12118_bhdgx9_00001,1,NaN,rue,du Foirail,12600,Mur-de-Barrez,673124.325185,6.416119e+06,1,NaN,rue,des Tilleuls,12600,Lacroix-Barrez,672606.00,6411666.96,4482.021189,0.250000,0.571429,0


In [169]:
training_df = training_df.drop(columns = ['_id','distanceXY', 'SimilarityScoreRue', 'SimilarityScoreCommune'])
training_df.columns


Index(['l_IdentifiantImmeuble', 'r_IdentifiantImmeuble',
       'l_NumeroVoieImmeuble', 'l_ComplementNumeroVoieImmeuble',
       'l_TypeVoieImmeuble', 'l_NomVoieImmeuble', 'l_CodePostalImmeuble',
       'l_CommuneImmeuble', 'l_CoordonneeImmeubleX', 'l_CoordonneeImmeubleY',
       'r_NumeroVoieImmeuble', 'r_ComplementNumeroVoieImmeuble',
       'r_TypeVoieImmeuble', 'r_NomVoieImmeuble', 'r_CodePostalImmeuble',
       'r_CommuneImmeuble', 'r_CoordonneeImmeubleX', 'r_CoordonneeImmeubleY',
       'label'],
      dtype='object')

In [170]:
cols = {'_id' : '_id', 'l_IdentifiantImmeuble' : 'id', 'r_IdentifiantImmeuble' : 'IdentifiantImmeuble',
       'l_NumeroVoieImmeuble' : 'numero', 'l_ComplementNumeroVoieImmeuble' : 'rep',
       'l_TypeVoieImmeuble' : 'TypeVoie', 'l_NomVoieImmeuble' : 'NomVoieReste', 'l_CodePostalImmeuble' : 'code_postal',
       'l_CommuneImmeuble' : 'nom_commune', 'l_CoordonneeImmeubleX' : 'x', 'l_CoordonneeImmeubleY' : 'y',
       'r_NumeroVoieImmeuble' : 'NumeroVoieImmeuble', 'r_ComplementNumeroVoieImmeuble' : 'ComplementNumeroVoieImmeuble',
       'r_TypeVoieImmeuble' : 'TypeVoieImmeuble', 'r_NomVoieImmeuble' : 'NomVoieImmeuble', 'r_CodePostalImmeuble' : 'CodePostalImmeuble',
       'r_CommuneImmeuble' : 'CommuneImmeuble', 'r_CoordonneeImmeubleX' : 'CoordonneeImmeubleX', 'r_CoordonneeImmeubleY' : 'CoordonneeImmeubleY'
       , 'label' : 'Label'}

training_df = training_df.rename(columns = cols)

In [171]:
labeled = training_df.copy()
labeled = labeled.dropna(subset=['id','IdentifiantImmeuble'])

labeled = labeled[~labeled['id'].duplicated()]
labeled = labeled[~labeled['IdentifiantImmeuble'].duplicated()]
labeled = labeled.fillna('')

labeled = labeled.astype({'id': 'string', 'numero' : 'float32' , 'rep' : 'string', 'TypeVoie': 'string', 'NomVoieReste': 'string', 'code_postal': 'int32',
                          'nom_commune': 'string', 'x': 'float32', 'y': 'float32', 'IdentifiantImmeuble': 'string', 'NumeroVoieImmeuble': 'float32',
                          'ComplementNumeroVoieImmeuble': 'string', 'TypeVoieImmeuble': 'string', 'NomVoieImmeuble': 'string',
                          'CodePostalImmeuble': 'int32', 'CommuneImmeuble': 'string', 'CoordonneeImmeubleX': 'float32',
                          'CoordonneeImmeubleY': 'float32', 'Label': 'int32'})

labeled = labeled[['id', 'numero', 'rep', 'TypeVoie', 'NomVoieReste', 'code_postal',
                  'nom_commune', 'x', 'y', 'IdentifiantImmeuble', 'NumeroVoieImmeuble',
                  'ComplementNumeroVoieImmeuble', 'TypeVoieImmeuble', 'NomVoieImmeuble',
                  'CodePostalImmeuble', 'CommuneImmeuble', 'CoordonneeImmeubleX',
                  'CoordonneeImmeubleY', 'Label']]

A = labeled.iloc[:, :len(labeled.columns)//2]
B = labeled.iloc[:, len(labeled.columns)//2:-1]
B['TypeVoieImmeuble'] = B['TypeVoieImmeuble'].apply(lambda x : unidecode(x).lower() if pd.isna(x) is False else '')

def complement_numero_voie(string) :
  try :
    string = string.lower()
  except :
    pass
  if string == 'bis' :
    string = 'b'
  if string == 'ter' :
    string = 't'
  if string == 'quater' :
    string = 'q'
  return string

A['rep'] = A['rep'].apply(complement_numero_voie)
B['ComplementNumeroVoieImmeuble'] = B['ComplementNumeroVoieImmeuble'].apply(complement_numero_voie)
ban = A.copy()
ipe = B.copy()
labeled = pd.concat([ban,ipe,labeled.iloc[:,-1]],axis=1)
nom_colonnes = {'id': 'ltable_id','numero' : 'ltable_NumeroVoieImmeuble','rep' : 'ltable_ComplementNumeroVoieImmeuble','TypeVoie' : 'ltable_TypeVoieImmeuble','NomVoieReste' : 'ltable_NomVoieImmeuble','code_postal' : 'ltable_CodePostalImmeuble','nom_commune' : 'ltable_CommuneImmeuble','x' : 'ltable_CoordonneeImmeubleX','y' : 'ltable_CoordonneeImmeubleY', 'IdentifiantImmeuble' : 'rtable_id', 'NumeroVoieImmeuble' : 'rtable_NumeroVoieImmeuble', 'ComplementNumeroVoieImmeuble' : 'rtable_ComplementNumeroVoieImmeuble', 'TypeVoieImmeuble' : 'rtable_TypeVoieImmeuble', 'NomVoieImmeuble' : 'rtable_NomVoieImmeuble', 'CodePostalImmeuble' : 'rtable_CodePostalImmeuble', 'CommuneImmeuble': 'rtable_CommuneImmeuble', 'CoordonneeImmeubleX' : 'rtable_CoordonneeImmeubleX', 'CoordonneeImmeubleY' : 'rtable_CoordonneeImmeubleY', 'Label' : 'label'}
labeled = labeled.rename(columns = nom_colonnes)


labeled['ltable_TypeVoieImmeuble'] = labeled['ltable_TypeVoieImmeuble'].apply(lambda x : unidecode(x).lower() if pd.isna(x) is False else '')
nom_colonnes = {'id': 'IdentifiantImmeuble','numero' : 'NumeroVoieImmeuble','rep' : 'ComplementNumeroVoieImmeuble','TypeVoie' : 'TypeVoieImmeuble','NomVoieReste' : 'NomVoieImmeuble','code_postal' : 'CodePostalImmeuble','nom_commune' : 'CommuneImmeuble','x' : 'CoordonneeImmeubleX','y' : 'CoordonneeImmeubleY'}
A = A.rename(columns=nom_colonnes)
labeled['_id'] = range(len(labeled))

#We need to set key for each table to create metadata so that we can continue the next step
em.set_key(labeled,'_id')
em.set_fk_ltable(labeled, 'ltable_id')
em.set_fk_rtable(labeled, 'rtable_id')
em.set_key(A, 'IdentifiantImmeuble')
em.set_key(B, 'IdentifiantImmeuble')
em.set_ltable(labeled, A)
em.set_rtable(labeled, B)

G = em.sample_table(labeled, labeled.shape[0])

#Customize the calculation of distance between 2 tuples
def real_distance(ltuple,rtuple):
  return np.sqrt((ltuple['CoordonneeImmeubleX']-rtuple['CoordonneeImmeubleX'])**2+(ltuple['CoordonneeImmeubleY']-rtuple['CoordonneeImmeubleY'])**2)

match_f = em.get_features_for_matching(A,B, validate_inferred_attr_types = False)

em.add_blackbox_feature(match_f,'real_distance',real_distance)
# we only keep relevant features
#feature_to_remain = [6,13,23,26,38,54]
feature_to_remain = [6,13,23,26,30,32,33,38,42,44,45,54]
match_f.drop([i for i in range(match_f.shape[0]) if i not in feature_to_remain],inplace=True)
match_f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
6,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,NumeroVoieImmeuble,NumeroVoieImmeuble,None,None,exact_match,<function NumeroVoieImmeuble_NumeroVoieImmeuble_exm at 0x7b55e4285d80>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
13,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,ComplementNumeroVoieImmeuble,ComplementNumeroVoieImmeuble,None,None,monge_elkan,<function ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel at 0x7b55e4284f70>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
23,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,TypeVoieImmeuble,TypeVoieImmeuble,None,None,lev_sim,<function TypeVoieImmeuble_TypeVoieImmeuble_lev_sim at 0x7b55e3ff5ea0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
26,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,NomVoieImmeuble,NomVoieImmeuble,qgm_3,qgm_3,jaccard,<function NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3 at 0x7b55cc5da290>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
30,NomVoieImmeuble_NomVoieImmeuble_lev_dist,NomVoieImmeuble,NomVoieImmeuble,None,None,lev_dist,<function NomVoieImmeuble_NomVoieImmeuble_lev_dist at 0x7b55cc5bf0a0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
32,NomVoieImmeuble_NomVoieImmeuble_nmw,NomVoieImmeuble,NomVoieImmeuble,None,None,needleman_wunsch,<function NomVoieImmeuble_NomVoieImmeuble_nmw at 0x7b55cc5be290>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
33,NomVoieImmeuble_NomVoieImmeuble_sw,NomVoieImmeuble,NomVoieImmeuble,None,None,smith_waterman,<function NomVoieImmeuble_NomVoieImmeuble_sw at 0x7b55cc5be0e0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
38,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble,CommuneImmeuble,qgm_3,qgm_3,jaccard,<function CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3 at 0x7b55cc5bf6d0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
42,CommuneImmeuble_CommuneImmeuble_lev_dist,CommuneImmeuble,CommuneImmeuble,None,None,lev_dist,<function CommuneImmeuble_CommuneImmeuble_lev_dist at 0x7b55cc5bf760>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
44,CommuneImmeuble_CommuneImmeuble_nmw,CommuneImmeuble,CommuneImmeuble,None,None,needleman_wunsch,<function CommuneImmeuble_CommuneImmeuble_nmw at 0x7b55cc5bfbe0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [172]:
H_train = em.extract_feature_vecs(G, feature_table=match_f , attrs_after=['label'])
dt = em.RFMatcher(n_estimators=300, max_depth = 15)

train_test = em.split_train_test(H_train, train_proportion=0.7)
devel_set = train_test['train']
eval_set = train_test['test']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


In [173]:
dt.fit(table=devel_set, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], target_attr='label')
pred_table = dt.predict(table=eval_set, exclude_attrs=['_id', 'ltable_id', 'rtable_id','label'], target_attr='predicted_labels', append=True, inplace=True)
pred_table


,_id,ltable_id,rtable_id,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,NomVoieImmeuble_NomVoieImmeuble_lev_dist,NomVoieImmeuble_NomVoieImmeuble_nmw,NomVoieImmeuble_NomVoieImmeuble_sw,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble_CommuneImmeuble_lev_dist,CommuneImmeuble_CommuneImmeuble_nmw,CommuneImmeuble_CommuneImmeuble_sw,real_distance,label,predicted_labels
6432,6015,IMB/12248/X/007R,12125_0d7w5m_00020,1,1.0,1.0,0.208333,9.0,1.0,4.0,0.025000,18.0,0.0,3.0,4465.711446,0,0
3134,3125,IMB/12248/X/001J,12248_0090_00050,1,1.0,1.0,0.727273,1.0,16.0,16.0,0.730769,3.0,16.0,18.0,12.556280,1,1
5906,5597,IMB/12300/X/04NB,12300_0640_00006,0,1.0,1.0,1.000000,0.0,11.0,11.0,1.000000,0.0,24.0,24.0,7.567405,0,0
1748,1742,IMB/12164/X/007B,12164_0003_00038,1,1.0,1.0,1.000000,0.0,10.0,10.0,1.000000,0.0,13.0,13.0,0.000000,1,1
5254,4995,IMB/12131/X/004E,12131_0556_00056,0,1.0,1.0,1.000000,0.0,8.0,8.0,1.000000,0.0,11.0,11.0,120.945817,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2852,2845,IMB/12013/X/01CJ,12013_0123_00021,1,1.0,1.0,0.647059,1.0,11.0,11.0,1.000000,0.0,5.0,5.0,6.768782,1,1
5666,5371,IMB/12211/X/003E,12211_t5h9by_03659,0,1.0,1.0,1.000000,0.0,14.0,14.0,0.730769,3.0,16.0,18.0,62.112077,0,0
490,490,IMB/12203/X/004G,12203_a029_00005,1,1.0,1.0,1.000000,0.0,11.0,11.0,1.000000,0.0,21.0,21.0,10.991652,1,1
2304,2298,IMB/12049/X/008P,12049_b332_00002,1,1.0,1.0,0.750000,1.0,14.0,15.0,1.000000,0.0,7.0,7.0,111.726525,1,1


In [174]:
eval_summary = em.eval_matches(pred_table, 'label', 'predicted_labels')
eval_summary

OrderedDict([('prec_numerator', 1145.0),
             ('prec_denominator', 1147.0),
             ('precision', 0.998256320836966),
             ('recall_numerator', 1145.0),
             ('recall_denominator', 1145.0),
             ('recall', 1.0),
             ('f1', 0.99912739965096),
             ('pred_pos_num', 1147.0),
             ('false_pos_num', 2.0),
             ('false_pos_ls',
              [('IMB/12220/X/00A6', '12122_0055_00005'),
               ('IMB/12300/X/03PZ', '12300_2760_00005')]),
             ('pred_neg_num', 868.0),
             ('false_neg_num', 0.0),
             ('false_neg_ls', [])])

### Model Evaluation

In [175]:
annote = pd.read_csv('/content/dataset_annote_V3.csv')

annote = annote.dropna(subset=['id','IdentifiantImmeuble'])


annote = annote.iloc[:-11,:]

annote = annote[~annote['id'].duplicated()]
annote = annote[~annote['IdentifiantImmeuble'].duplicated()]
annote = annote.fillna('')

annote = annote.astype({'id': 'string', 'numero' : 'float32' , 'rep' : 'string', 'TypeVoie': 'string', 'NomVoieReste': 'string', 'code_postal': 'int32',
                          'nom_commune': 'string', 'x': 'float32', 'y': 'float32', 'IdentifiantImmeuble': 'string', 'NumeroVoieImmeuble': 'float32',
                          'ComplementNumeroVoieImmeuble': 'string', 'TypeVoieImmeuble': 'string', 'NomVoieImmeuble': 'string',
                          'CodePostalImmeuble': 'int32', 'CommuneImmeuble': 'string', 'CoordonneeImmeubleX': 'float32',
                          'CoordonneeImmeubleY': 'float32', 'Label': 'int32'})

A = annote.iloc[:, :len(annote.columns)//2]
B = annote.iloc[:, len(annote.columns)//2:-1]



def complement_numero_voie(string) :
  try :
    string = string.lower()
  except :
    pass
  if string == 'bis' :
    string = 'b'
  if string == 'ter' :
    string = 't'
  if string == 'quater' :
    string = 'q'
  return string

A['rep'] = A['rep'].apply(complement_numero_voie)
B['ComplementNumeroVoieImmeuble'] = B['ComplementNumeroVoieImmeuble'].apply(complement_numero_voie)

ban = A.copy()
ipe = B.copy()
labeled = pd.concat([ban,ipe,annote.iloc[:,-1]],axis=1)
nom_colonnes = {'id': 'ltable_id','numero' : 'ltable_NumeroVoieImmeuble','rep' : 'ltable_ComplementNumeroVoieImmeuble','TypeVoie' : 'ltable_TypeVoieImmeuble','NomVoieReste' : 'ltable_NomVoieImmeuble','code_postal' : 'ltable_CodePostalImmeuble','nom_commune' : 'ltable_CommuneImmeuble','x' : 'ltable_CoordonneeImmeubleX','y' : 'ltable_CoordonneeImmeubleY', 'IdentifiantImmeuble' : 'rtable_id', 'NumeroVoieImmeuble' : 'rtable_NumeroVoieImmeuble', 'ComplementNumeroVoieImmeuble' : 'rtable_ComplementNumeroVoieImmeuble', 'TypeVoieImmeuble' : 'rtable_TypeVoieImmeuble', 'NomVoieImmeuble' : 'rtable_NomVoieImmeuble', 'CodePostalImmeuble' : 'rtable_CodePostalImmeuble', 'CommuneImmeuble': 'rtable_CommuneImmeuble', 'CoordonneeImmeubleX' : 'rtable_CoordonneeImmeubleX', 'CoordonneImmeubleY' : 'rtable_CoordonneeImmeubleY', 'Label' : 'label'}
labeled = labeled.rename(columns = nom_colonnes)

labeled['ltable_TypeVoieImmeuble'] = labeled['ltable_TypeVoieImmeuble'].apply(lambda x : unidecode(x).lower() if pd.isna(x) is False else '')
labeled['rtable_TypeVoieImmeuble'] = labeled['rtable_TypeVoieImmeuble'].apply(lambda x : unidecode(x).lower() if pd.isna(x) is False else '')


nom_colonnes = {'id': 'IdentifiantImmeuble','numero' : 'NumeroVoieImmeuble','rep' : 'ComplementNumeroVoieImmeuble','TypeVoie' : 'TypeVoieImmeuble','NomVoieReste' : 'NomVoieImmeuble','code_postal' : 'CodePostalImmeuble','nom_commune' : 'CommuneImmeuble','x' : 'CoordonneeImmeubleX','y' : 'CoordonneeImmeubleY'}
A = A.rename(columns=nom_colonnes)

labeled['_id'] = range(len(labeled))

#We need to set key for each table to create metadata so that we can continue the next step
em.set_key(labeled,'_id')
em.set_fk_ltable(labeled, 'ltable_id')
em.set_fk_rtable(labeled, 'rtable_id')
em.set_key(A, 'IdentifiantImmeuble')
em.set_key(B, 'IdentifiantImmeuble')
em.set_ltable(labeled, A)
em.set_rtable(labeled, B)

labeled


,ltable_id,ltable_NumeroVoieImmeuble,ltable_ComplementNumeroVoieImmeuble,ltable_TypeVoieImmeuble,ltable_NomVoieImmeuble,ltable_CodePostalImmeuble,ltable_CommuneImmeuble,ltable_CoordonneeImmeubleX,ltable_CoordonneeImmeubleY,rtable_id,rtable_NumeroVoieImmeuble,rtable_ComplementNumeroVoieImmeuble,rtable_TypeVoieImmeuble,rtable_NomVoieImmeuble,rtable_CodePostalImmeuble,rtable_CommuneImmeuble,rtable_CoordonneeImmeubleX,CoordonneeImmeubleY,label,_id
0,12002_0puwwe_00001,1.0,,plan,du Foirail,12520,Aguessac,707758.8750,6339741.0,IMB/12001/X/0001,4.0,,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.1250,6361511.0,0,0
1,12002_ntujmi_00001,1.0,,avenue,de la Brezegue,12520,Aguessac,707676.5000,6339929.5,IMB/12001/X/0006,3.0,,chemin,de Campan,12630,Agen-d'Aveyron,673813.5625,6361964.0,0,1
2,12002_ntujmi_00002,2.0,,avenue,de la Brezegue,12520,Aguessac,707691.1250,6339931.5,IMB/12001/X/0007,9.0,,chemin,de Campan,12630,Agen-d'Aveyron,673709.7500,6361977.5,0,2
3,12002_ntujmi_00003,3.0,,avenue,de la Brezegue,12520,Aguessac,707569.0625,6339992.0,IMB/12001/X/000H,14.0,,chemin,de Cantaserp,12630,Agen-d'Aveyron,673441.2500,6362122.0,0,3
4,12002_ntujmi_00004,4.0,,avenue,de la Brezegue,12520,Aguessac,707643.8125,6339963.5,IMB/12001/X/000I,16.0,,chemin,de Cantaserp,12630,Agen-d'Aveyron,673431.8750,6362076.0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,12299_0024_00024,24.0,,chemin,de la Carral,12430,Villefranche-de-Panat,676497.2500,6331368.0,IMB/12008/X/0001,1.0,,chemin,de la Carrière,12390,Anglars-St-Félix,641301.2500,6369760.0,0,168
184,12008_0060_00002,2.0,,chemin,de la Carrière,12390,Anglars-Saint-Félix,641345.3125,6369750.5,IMB/12008/X/0002,2.0,,chemin,de la Carrière,12390,Anglars-St-Félix,641353.2500,6369745.5,1,169
185,12008_0045_00001,1.0,,chemin,des Bosquets,12390,Anglars-Saint-Félix,641467.0000,6369919.5,IMB/12008/X/000H,1.0,,chemin,des Bosquets,12390,Anglars-St-Félix,641455.5000,6369919.0,1,170
186,12300_2470_00059_ter,59.0,t,avenue,du Quercy,12200,Villefranche-de-Rouergue,622621.1250,6362129.5,IMB/12300/X/00YI,59.0,t,avenue,du Quercy,12200,Villefranche-de-Rouergue,622595.5000,6362158.0,1,171


In [176]:
H = em.extract_feature_vecs(labeled, feature_table=match_f)
H

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


,_id,ltable_id,rtable_id,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,NomVoieImmeuble_NomVoieImmeuble_lev_dist,NomVoieImmeuble_NomVoieImmeuble_nmw,NomVoieImmeuble_NomVoieImmeuble_sw,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble_CommuneImmeuble_lev_dist,CommuneImmeuble_CommuneImmeuble_nmw,CommuneImmeuble_CommuneImmeuble_sw,real_distance
0,0,12002_0puwwe_00001,IMB/12001/X/0001,0,1.0,0.142857,0.032258,15.0,-5.0,3.0,0.083333,11.0,-3.0,2.0,39802.076335
1,1,12002_ntujmi_00001,IMB/12001/X/0006,0,1.0,0.166667,0.125000,10.0,-1.0,4.0,0.083333,11.0,-3.0,2.0,40400.714429
2,2,12002_ntujmi_00002,IMB/12001/X/0007,0,1.0,0.166667,0.125000,10.0,-1.0,4.0,0.083333,11.0,-3.0,2.0,40506.295349
3,3,12002_ntujmi_00003,IMB/12001/X/000H,0,1.0,0.166667,0.111111,9.0,3.0,4.0,0.083333,11.0,-3.0,2.0,40674.863073
4,4,12002_ntujmi_00004,IMB/12001/X/000I,0,1.0,0.166667,0.111111,9.0,3.0,4.0,0.083333,11.0,-3.0,2.0,40735.970883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,168,12299_0024_00024,IMB/12008/X/0001,0,1.0,1.000000,0.500000,4.0,8.0,10.0,0.000000,17.0,-1.0,2.0,52083.625834
184,169,12008_0060_00002,IMB/12008/X/0002,1,1.0,1.000000,1.000000,0.0,14.0,14.0,0.695652,3.0,13.0,13.0,9.381040
185,170,12008_0045_00001,IMB/12008/X/000H,1,1.0,1.000000,1.000000,0.0,12.0,12.0,0.695652,3.0,13.0,13.0,11.510864
186,171,12300_2470_00059_ter,IMB/12300/X/00YI,1,1.0,1.000000,1.000000,0.0,9.0,9.0,1.000000,0.0,24.0,24.0,38.326109


In [177]:
pred_table = dt.predict(table=H, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], target_attr='predicted_labels', append=True, inplace=True)
result_df = pd.merge(pred_table, labeled[['_id', 'label']], on='_id', how='left')
result_df

,_id,ltable_id,rtable_id,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,NomVoieImmeuble_NomVoieImmeuble_lev_dist,NomVoieImmeuble_NomVoieImmeuble_nmw,NomVoieImmeuble_NomVoieImmeuble_sw,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble_CommuneImmeuble_lev_dist,CommuneImmeuble_CommuneImmeuble_nmw,CommuneImmeuble_CommuneImmeuble_sw,real_distance,predicted_labels,label
0,0,12002_0puwwe_00001,IMB/12001/X/0001,0,1.0,0.142857,0.032258,15.0,-5.0,3.0,0.083333,11.0,-3.0,2.0,39802.076335,0,0
1,1,12002_ntujmi_00001,IMB/12001/X/0006,0,1.0,0.166667,0.125000,10.0,-1.0,4.0,0.083333,11.0,-3.0,2.0,40400.714429,0,0
2,2,12002_ntujmi_00002,IMB/12001/X/0007,0,1.0,0.166667,0.125000,10.0,-1.0,4.0,0.083333,11.0,-3.0,2.0,40506.295349,0,0
3,3,12002_ntujmi_00003,IMB/12001/X/000H,0,1.0,0.166667,0.111111,9.0,3.0,4.0,0.083333,11.0,-3.0,2.0,40674.863073,0,0
4,4,12002_ntujmi_00004,IMB/12001/X/000I,0,1.0,0.166667,0.111111,9.0,3.0,4.0,0.083333,11.0,-3.0,2.0,40735.970883,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,168,12299_0024_00024,IMB/12008/X/0001,0,1.0,1.000000,0.500000,4.0,8.0,10.0,0.000000,17.0,-1.0,2.0,52083.625834,0,0
169,169,12008_0060_00002,IMB/12008/X/0002,1,1.0,1.000000,1.000000,0.0,14.0,14.0,0.695652,3.0,13.0,13.0,9.381040,1,1
170,170,12008_0045_00001,IMB/12008/X/000H,1,1.0,1.000000,1.000000,0.0,12.0,12.0,0.695652,3.0,13.0,13.0,11.510864,1,1
171,171,12300_2470_00059_ter,IMB/12300/X/00YI,1,1.0,1.000000,1.000000,0.0,9.0,9.0,1.000000,0.0,24.0,24.0,38.326109,1,1


In [178]:
true_positive = len(result_df[(result_df['predicted_labels'] == 1) & (result_df['label'] == 1)])
false_positive = len(result_df[(result_df['predicted_labels'] == 1) & (result_df['label'] == 0)])
true_negative = len(result_df[(result_df['predicted_labels'] == 0) & (result_df['label'] == 0)])
false_negative = len(result_df[(result_df['predicted_labels'] == 0) & (result_df['label'] == 1)])

accuracy = (true_positive + true_negative)/len(result_df)
recall = true_positive/(true_positive + false_negative)
precision = true_positive/(true_positive + false_positive)

print('accuracy : ',accuracy*100, ' %' )
print('recall : ',recall*100, ' %' )
print('precision : ',precision*100, ' %' )

accuracy :  95.95375722543352  %
recall :  100.0  %
precision :  92.3913043478261  %


# Prediction

Dans cette partie, nous utilisons le modèle entrainé afin de prédire si les adresses sont similaires ou non.

In [ ]:
#si vous avez enregistré les dataframes de la section précédente sous forme de CSV pour les réutiliser, vous pouvez les charger ici en précisant le chemin des CSV.
ban_reduced = pd.read_csv('/content/ban_reduced.csv')
ipe_reduced = pd.read_csv('/content/ipe_reduced.csv')

A = ipe_reduced.copy()
B = ban_reduced.copy()

#A.rename(columns={'IdentifiantImmeuble': 'id'}, inplace=True)
#B.rename(columns={'IdentifiantImmeuble': 'id'}, inplace=True)

em.set_key(A, 'IdentifiantImmeuble')
em.set_key(B, 'IdentifiantImmeuble')


shape_A = A.shape
shape_B = B.shape
print(shape_A)
print(shape_B)

(283831, 10)
(152683, 10)


In [ ]:
def complement_numero_voie(string) :
  try :
    string = string.lower()
  except :
    pass
  if string == 'bis' :
    string = 'b'
  if string == 'ter' :
    string = 't'
  if string == 'quater' :
    string = 'q'
  return string

A['ComplementNumeroVoieImmeuble'] = A['ComplementNumeroVoieImmeuble'].apply(complement_numero_voie)
B['ComplementNumeroVoieImmeuble'] = B['ComplementNumeroVoieImmeuble'].apply(complement_numero_voie)



### Blocking

Comme pour l'algorithme 1 basé sur nos propres règles, nous définissons des colonnes pour lesquelles la correspondance doit être stricte entre les 2 adresses :
- NumeroVoieImmeuble
- ComplementNumeroVoieImmeuble
- TypeVoieImmeuble
- CodePostalImmeuble


In [ ]:
# define the blocker attributes to check and put in 1 column (because AttrEquivalenceBlocker can only handle 1 column comparison)


A['blocking_data'] = A['NumeroVoieImmeuble'].fillna('').astype(str)+ ' '  +  A['ComplementNumeroVoieImmeuble'].fillna('').astype(str) + ' ' + A['TypeVoieImmeuble'].fillna('').astype(str) + ' ' + A['CodePostalImmeuble'].fillna('').astype(str)
B['blocking_data'] = B['NumeroVoieImmeuble'].fillna('').astype(str)+ ' '  +  B['ComplementNumeroVoieImmeuble'].fillna('').astype(str) + ' ' + B['TypeVoieImmeuble'].fillna('').astype(str) + ' ' + B['CodePostalImmeuble'].fillna('').astype(str)

attributes = ['IdentifiantImmeuble','NumeroVoieImmeuble','ComplementNumeroVoieImmeuble','TypeVoieImmeuble','NomVoieImmeuble','CodePostalImmeuble','CommuneImmeuble','CoordonneeImmeubleX','CoordonneeImmeubleY']

ab = em.AttrEquivalenceBlocker()

# Use block_tables to apply blocking over two input tables.
C = ab.block_tables(A, B,
                    l_block_attr='blocking_data', r_block_attr='blocking_data',
                    l_output_attrs=attributes,
                    r_output_attrs=attributes,
                    l_output_prefix='ltable_', r_output_prefix='rtable_')

C

,_id,ltable_IdentifiantImmeuble,rtable_IdentifiantImmeuble,ltable_NumeroVoieImmeuble,ltable_ComplementNumeroVoieImmeuble,ltable_TypeVoieImmeuble,ltable_NomVoieImmeuble,ltable_CodePostalImmeuble,ltable_CommuneImmeuble,ltable_CoordonneeImmeubleX,ltable_CoordonneeImmeubleY,rtable_NumeroVoieImmeuble,rtable_ComplementNumeroVoieImmeuble,rtable_TypeVoieImmeuble,rtable_NomVoieImmeuble,rtable_CodePostalImmeuble,rtable_CommuneImmeuble,rtable_CoordonneeImmeubleX,rtable_CoordonneeImmeubleY
0,0,IMB/12001/X/0001,12001_0052_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06,4,NaN,impasse,Adrien Vezinhet,12630,Agen-d'Aveyron,673832.68,6362135.56
1,1,IMB/12001/X/0001,12001_0040_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06,4,NaN,impasse,Daude de Prades,12630,Agen-d'Aveyron,673699.44,6362045.91
2,2,IMB/12001/X/0001,12001_0083_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674471.55,6361516.82
3,3,IMB/12001/X/0001,12001_0067_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06,4,NaN,impasse,de la Cascade,12630,Agen-d'Aveyron,677171.63,6360381.52
4,4,IMB/12001/X/0001,12001_0069_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06,4,NaN,impasse,de la Croux,12630,Agen-d'Aveyron,674318.84,6362009.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133860,1133860,IMB/12307/X/00BB,12307_0018_00408,408,NaN,chemin,de Bellevue,12410,Curan,689708.742599,6.344055e+06,408,NaN,chemin,de Bellevue,12410,Curan,689727.62,6344090.54
1133861,1133861,IMB/12307/X/00BC,12307_0039_00064,64,NaN,impasse,du Claux,12410,Curan,688529.268230,6.344332e+06,64,NaN,impasse,du Claux,12410,Curan,688513.51,6344373.61
1133862,1133862,IMB/12307/X/00BG,12307_0039_00064,64,NaN,impasse,du Claux,12410,Curan,688560.571568,6.344372e+06,64,NaN,impasse,du Claux,12410,Curan,688513.51,6344373.61
1133863,1133863,IMB/12307/X/00BH,12307_0084_00349,349,NaN,chemin,de Martials,12410,Curan,689480.028156,6.347026e+06,349,NaN,chemin,de Martials,12410,Curan,689478.05,6347044.43


### Machine learning Prediction

In [124]:
H = em.extract_feature_vecs(C, feature_table=match_f)
H = em.impute_table(H, exclude_attrs=['_id', 'ltable_IdentifiantImmeuble', 'rtable_IdentifiantImmeuble'], strategy='mean')
H

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:09:23


,_id,ltable_IdentifiantImmeuble,rtable_IdentifiantImmeuble,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,NomVoieImmeuble_NomVoieImmeuble_lev_dist,NomVoieImmeuble_NomVoieImmeuble_nmw,NomVoieImmeuble_NomVoieImmeuble_sw,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble_CommuneImmeuble_lev_dist,CommuneImmeuble_CommuneImmeuble_nmw,CommuneImmeuble_CommuneImmeuble_sw,real_distance
0,0,IMB/12001/X/0001,12001_0052_00004,1.0,1.0,1.0,0.000000,17.0,-3.0,3.0,1.0,0.0,14.0,14.0,869.844613
1,1,IMB/12001/X/0001,12001_0040_00004,1.0,1.0,1.0,0.028571,14.0,-2.0,5.0,1.0,0.0,14.0,14.0,912.011652
2,2,IMB/12001/X/0001,12001_0083_00004,1.0,1.0,1.0,1.000000,0.0,19.0,19.0,1.0,0.0,14.0,14.0,33.955714
3,3,IMB/12001/X/0001,12001_0067_00004,1.0,1.0,1.0,0.250000,10.0,3.0,8.0,1.0,0.0,14.0,14.0,2957.680510
4,4,IMB/12001/X/0001,12001_0069_00004,1.0,1.0,1.0,0.222222,11.0,0.0,6.0,1.0,0.0,14.0,14.0,512.126756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133860,1133860,IMB/12307/X/00BB,12307_0018_00408,1.0,1.0,1.0,1.000000,0.0,11.0,11.0,1.0,0.0,5.0,5.0,40.513252
1133861,1133861,IMB/12307/X/00BC,12307_0039_00064,1.0,1.0,1.0,1.000000,0.0,8.0,8.0,1.0,0.0,5.0,5.0,44.155804
1133862,1133862,IMB/12307/X/00BG,12307_0039_00064,1.0,1.0,1.0,1.000000,0.0,8.0,8.0,1.0,0.0,5.0,5.0,47.087153
1133863,1133863,IMB/12307/X/00BH,12307_0084_00349,1.0,1.0,1.0,1.000000,0.0,11.0,11.0,1.0,0.0,5.0,5.0,18.890709


In [125]:
#dt.fit(table=devel_set, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], target_attr='label')
#pred_table = dt.predict(table=H, exclude_attrs=['_id', 'ltable_IdentifiantImmeuble', 'rtable_IdentifiantImmeuble','predicted_labels'], target_attr='predicted_labels', append=True, inplace=True)
pred_table = dt.predict(table=H, exclude_attrs=['_id', 'ltable_IdentifiantImmeuble', 'rtable_IdentifiantImmeuble'], target_attr='predicted_labels', append=True, inplace=True)
pred_table


,_id,ltable_IdentifiantImmeuble,rtable_IdentifiantImmeuble,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,NomVoieImmeuble_NomVoieImmeuble_lev_dist,NomVoieImmeuble_NomVoieImmeuble_nmw,NomVoieImmeuble_NomVoieImmeuble_sw,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble_CommuneImmeuble_lev_dist,CommuneImmeuble_CommuneImmeuble_nmw,CommuneImmeuble_CommuneImmeuble_sw,real_distance,predicted_labels
0,0,IMB/12001/X/0001,12001_0052_00004,1.0,1.0,1.0,0.000000,17.0,-3.0,3.0,1.0,0.0,14.0,14.0,869.844613,0
1,1,IMB/12001/X/0001,12001_0040_00004,1.0,1.0,1.0,0.028571,14.0,-2.0,5.0,1.0,0.0,14.0,14.0,912.011652,0
2,2,IMB/12001/X/0001,12001_0083_00004,1.0,1.0,1.0,1.000000,0.0,19.0,19.0,1.0,0.0,14.0,14.0,33.955714,1
3,3,IMB/12001/X/0001,12001_0067_00004,1.0,1.0,1.0,0.250000,10.0,3.0,8.0,1.0,0.0,14.0,14.0,2957.680510,0
4,4,IMB/12001/X/0001,12001_0069_00004,1.0,1.0,1.0,0.222222,11.0,0.0,6.0,1.0,0.0,14.0,14.0,512.126756,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133860,1133860,IMB/12307/X/00BB,12307_0018_00408,1.0,1.0,1.0,1.000000,0.0,11.0,11.0,1.0,0.0,5.0,5.0,40.513252,1
1133861,1133861,IMB/12307/X/00BC,12307_0039_00064,1.0,1.0,1.0,1.000000,0.0,8.0,8.0,1.0,0.0,5.0,5.0,44.155804,1
1133862,1133862,IMB/12307/X/00BG,12307_0039_00064,1.0,1.0,1.0,1.000000,0.0,8.0,8.0,1.0,0.0,5.0,5.0,47.087153,1
1133863,1133863,IMB/12307/X/00BH,12307_0084_00349,1.0,1.0,1.0,1.000000,0.0,11.0,11.0,1.0,0.0,5.0,5.0,18.890709,1


In [128]:
importances = dt.clf.feature_importances_
# Get indices of features sorted by importance
indices = np.argsort(importances)[::-1]

# Print feature ranking
print("Feature ranking:")
for f in range(6):
    print("%d. feature %str (%f)" % (f + 1, devel_set.columns[3:-1][indices[f]], importances[indices[f]]))

Feature ranking:
1. feature NumeroVoieImmeuble_NumeroVoieImmeuble_exmtr (0.412568)
2. feature NomVoieImmeuble_NomVoieImmeuble_lev_disttr (0.181912)
3. feature NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3tr (0.130440)
4. feature NomVoieImmeuble_NomVoieImmeuble_nmwtr (0.083958)
5. feature real_distancetr (0.067883)
6. feature NomVoieImmeuble_NomVoieImmeuble_swtr (0.067861)


Si une même adresse d'une table a été matchée plusieurs fois (avec plusieurs adresses de l'autre table), on garde seulement le match qui correspond à la distance la plus faible entre les coordonnées, c'est à dire les 2 adresses les plus proches spatialement

In [ ]:
match_table = pred_table[pred_table['predicted_labels'] == 1]
df_sorted = match_table.sort_values(by='real_distance')

# We keep only the first occurrence of each address
match_table1 = df_sorted.drop_duplicates(subset='ltable_IdentifiantImmeuble', keep='first')
match_table2 = match_table1.drop_duplicates(subset='rtable_IdentifiantImmeuble', keep='first')
print(len(match_table2))

In [129]:
print('We found', len(match_table2), 'matching adresses')

We found 72308 matching adresses
